# Requirements

In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
!pip install polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 17.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import polars as pl

In [4]:
import re

# Reading Data

In [5]:
dft = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet')

# Analysis

In [6]:
pl.read_parquet_schema('/kaggle/input/aeroclub-recsys-2025/train.parquet')

Schema([('Id', Int64),
        ('bySelf', Boolean),
        ('companyID', Int64),
        ('corporateTariffCode', Int64),
        ('frequentFlyer', String),
        ('nationality', Int64),
        ('isAccess3D', Boolean),
        ('isVip', Boolean),
        ('legs0_arrivalAt', String),
        ('legs0_departureAt', String),
        ('legs0_duration', String),
        ('legs0_segments0_aircraft_code', String),
        ('legs0_segments0_arrivalTo_airport_city_iata', String),
        ('legs0_segments0_arrivalTo_airport_iata', String),
        ('legs0_segments0_baggageAllowance_quantity', Float64),
        ('legs0_segments0_baggageAllowance_weightMeasurementType', Float64),
        ('legs0_segments0_cabinClass', Float64),
        ('legs0_segments0_departureFrom_airport_iata', String),
        ('legs0_segments0_duration', String),
        ('legs0_segments0_flightNumber', String),
        ('legs0_segments0_marketingCarrier_code', String),
        ('legs0_segments0_operatingCarrier_code', Str

In [7]:
dft

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64
0,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""","""02:40:00""","""YK2""","""KJA""","""KJA""",1.0,0.0,1.0,"""TLK""","""02:40:00""","""216""","""KV""","""KV""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,370.0,16884.0,1,0
1,true,57323,123,"""S7/SU/UT""",36,true,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,51125.0,0,1
2,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf696

# Cleaning Data & Feature Engineering

In [8]:
dft= dft.drop('bySelf','sex','legs0_segments0_cabinClass','nationality','legs0_segments0_marketingCarrier_code','legs0_segments2_marketingCarrier_code','legs0_segments3_marketingCarrier_code','legs1_segments0_marketingCarrier_code','legs1_segments2_marketingCarrier_code','legs1_segments3_marketingCarrier_code')

In [9]:
dft = dft.drop('legs1_segments0_duration')

In [10]:
dft = dft.with_columns(
    TotalPrice = pl.col("totalPrice") + pl.col("taxes"))

In [11]:
dft['TotalPrice']

TotalPrice
f64
17254.0
53365.0
55935.0
84120.0
88310.0
…
36290.0
33220.0
29920.0


In [12]:
dft = dft.drop('legs0_segments0_duration','legs0_segments1_duration','legs0_segments2_duration','legs0_segments3_duration')

In [13]:
dft = dft.drop('legs1_segments1_duration','legs1_segments2_duration','legs1_segments3_duration')

In [14]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-06-15T15:40:00""","""2024-06-15T16:20:00""","""02:40:00""","""2024-07-09T09:45:00""","""2024-07-09T14:20:00""","""02:35:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
…,…,…,…,…,…
"""2024-11-05T20:50:00""","""2024-11-06T09:45:00""","""10:55:00""","""2024-11-09T19:25:00""","""2024-11-10T08:10:00""","""14:45:00"""
"""2024-11-05T00:20:00""","""2024-11-05T20:00:00""","""17:40:00""","""2024-11-09T21:10:00""","""2024-11-10T08:35:00""","""13:25:00"""
"""2024-11-05T00:20:00""","""2024-11-05T20:00:00""","""17:40:00""","""2024-11-09T19:25:00""","""2024-11-10T08:10:00""","""14:45:00"""


In [15]:
dft= dft.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration').str.replace(r"T", "  "))

In [16]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-06-15 15:40:00""","""2024-06-15 16:20:00""","""02:40:00""","""2024-07-09 09:45:00""","""2024-07-09 14:20:00""","""02:35:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
…,…,…,…,…,…
"""2024-11-05 20:50:00""","""2024-11-06 09:45:00""","""10:55:00""","""2024-11-09 19:25:00""","""2024-11-10 08:10:00""","""14:45:00"""
"""2024-11-05 00:20:00""","""2024-11-05 20:00:00""","""17:40:00""","""2024-11-09 21:10:00""","""2024-11-10 08:35:00""","""13:25:00"""
"""2024-11-05 00:20:00""","""2024-11-05 20:00:00""","""17:40:00""","""2024-11-09 19:25:00""","""2024-11-10 08:10:00""","""14:45:00"""


In [17]:
dft= dft.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt').str.to_datetime("%Y-%m-%d %H:%M:%S"))

In [18]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt']

legs0_departureAt,legs0_arrivalAt,legs1_departureAt,legs1_arrivalAt
datetime[μs],datetime[μs],datetime[μs],datetime[μs]
2024-06-15 15:40:00,2024-06-15 16:20:00,2024-07-09 09:45:00,2024-07-09 14:20:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
…,…,…,…
2024-11-05 20:50:00,2024-11-06 09:45:00,2024-11-09 19:25:00,2024-11-10 08:10:00
2024-11-05 00:20:00,2024-11-05 20:00:00,2024-11-09 21:10:00,2024-11-10 08:35:00
2024-11-05 00:20:00,2024-11-05 20:00:00,2024-11-09 19:25:00,2024-11-10 08:10:00


In [19]:
dft['legs0_duration',
'legs1_duration']

legs0_duration,legs1_duration
str,str
"""02:40:00""","""02:35:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
…,…
"""10:55:00""","""14:45:00"""
"""17:40:00""","""13:25:00"""
"""17:40:00""","""14:45:00"""


In [20]:
dft = dft.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.replace_all(r"\.\d{2}", "")
)

In [21]:
dft=dft.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.to_time("%H:%M:%S"))

In [22]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
datetime[μs],datetime[μs],time,datetime[μs],datetime[μs],time
2024-06-15 15:40:00,2024-06-15 16:20:00,02:40:00,2024-07-09 09:45:00,2024-07-09 14:20:00,02:35:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
…,…,…,…,…,…
2024-11-05 20:50:00,2024-11-06 09:45:00,10:55:00,2024-11-09 19:25:00,2024-11-10 08:10:00,14:45:00
2024-11-05 00:20:00,2024-11-05 20:00:00,17:40:00,2024-11-09 21:10:00,2024-11-10 08:35:00,13:25:00
2024-11-05 00:20:00,2024-11-05 20:00:00,17:40:00,2024-11-09 19:25:00,2024-11-10 08:10:00,14:45:00


In [23]:
dftt = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet')

In [24]:
dftt= dftt.drop('bySelf','sex','legs0_segments0_cabinClass','nationality','legs0_segments0_marketingCarrier_code','legs0_segments2_marketingCarrier_code','legs0_segments3_marketingCarrier_code','legs1_segments0_marketingCarrier_code','legs1_segments2_marketingCarrier_code','legs1_segments3_marketingCarrier_code',)

In [25]:
dftt = dftt.with_columns(
    TotalPrice = pl.col("totalPrice") + pl.col("taxes"))

In [26]:
dftt = dftt.drop('legs1_segments0_duration')

In [27]:
dftt = dftt.drop('legs0_segments0_duration','legs0_segments1_duration','legs0_segments2_duration','legs0_segments3_duration')

In [28]:
dftt = dftt.drop('legs1_segments1_duration','legs1_segments2_duration','legs1_segments3_duration')

In [29]:
dftt['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T08:25:00""","""2024-12-19T12:45:00""","""02:20:00""","""2024-12-21T12:00:00""","""2024-12-21T12:35:00""","""02:35:00"""
"""2024-12-19T08:25:00""","""2024-12-19T12:45:00""","""02:20:00""","""2024-12-21T12:00:00""","""2024-12-21T12:35:00""","""02:35:00"""
…,…,…,…,…,…
"""2025-01-08T09:05:00""","""2025-01-08T12:50:00""","""02:45:00""",null,null,null
"""2025-01-08T21:25:00""","""2025-01-09T01:10:00""","""02:45:00""",null,null,null
"""2025-01-08T21:25:00""","""2025-01-09T01:10:00""","""02:45:00""",null,null,null


In [30]:
dftt= dftt.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration').str.replace(r"T", "  "))

In [31]:
dftt= dftt.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt').str.to_datetime("%Y-%m-%d %H:%M:%S"))

In [32]:
dftt = dftt.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.replace_all(r"\.\d{2}", "")
)

In [33]:
dftt=dftt.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.to_time("%H:%M:%S"))

In [34]:
dftt['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
datetime[μs],datetime[μs],time,datetime[μs],datetime[μs],time
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 08:25:00,2024-12-19 12:45:00,02:20:00,2024-12-21 12:00:00,2024-12-21 12:35:00,02:35:00
2024-12-19 08:25:00,2024-12-19 12:45:00,02:20:00,2024-12-21 12:00:00,2024-12-21 12:35:00,02:35:00
…,…,…,…,…,…
2025-01-08 09:05:00,2025-01-08 12:50:00,02:45:00,null,null,null
2025-01-08 21:25:00,2025-01-09 01:10:00,02:45:00,null,null,null
2025-01-08 21:25:00,2025-01-09 01:10:00,02:45:00,null,null,null


In [ ]:
def preprocess(df: pl.DataFrame) -> pl.DataFrame:
    datetime_cols = ["legs0_departureAt", "legs0_arrivalAt",
                     "legs1_departureAt", "legs1_arrivalAt"]

    for col in datetime_cols:
        # Ensure datetime type
        df = df.with_columns(
            pl.col(col).cast(pl.Datetime(time_unit="us"))
        )

        # Basic datetime features
        df = df.with_columns([
            pl.col(col).dt.hour().alias(f"{col}_hour"),
            pl.col(col).dt.weekday().alias(f"{col}_dow"),
            pl.col(col).dt.month().alias(f"{col}_month"),
            (pl.col(col).dt.weekday() >= 5).cast(pl.Int8).alias(f"{col}_is_weekend"),
        ])

        # Cyclical hour encoding
        df = df.with_columns([
            (pl.col(f"{col}_hour") * (2 * np.pi) / 24).sin().alias(f"{col}_hour_sin"),
            (pl.col(f"{col}_hour") * (2 * np.pi) / 24).cos().alias(f"{col}_hour_cos"),
        ])

    # Duration to minutes
    duration_cols = ["legs0_duration", "legs1_duration"]
    for col in duration_cols:
    # If already a Duration, just convert to minutes
        if df[col].dtype == pl.Duration(time_unit="us"):
            df = df.with_columns(
                (pl.col(col).dt.seconds() / 60).alias(col)
            )
        # If it's a Time (HH:MM:SS)
        elif df[col].dtype == pl.Time:
            df = df.with_columns(
                (pl.col(col).dt.hour() * 60 +
                 pl.col(col).dt.minute() +
                 pl.col(col).dt.second() / 60).alias(col)
            )
        # If it's a string like "02:40:00"
        else:
            df = df.with_columns(
                (pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.hour() * 60 +
                 pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.minute() +
                 pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.second() / 60
                ).alias(col)
            )
   
    return df

In [ ]:
print("Preprocessing...")
dft = preprocess(dft)
dftt = preprocess(dftt)

In [ ]:
dft = dft.with_columns(
    pl.col("legs1_duration","legs0_duration").abs()
)
dftt = dftt.with_columns(
    pl.col("legs1_duration","legs0_duration").abs()
)

# Feature Selection

In [35]:
y = dft['selected']

In [36]:
x = dft['Id','TotalPrice','searchRoute','corporateTariffCode','profileId','companyID',
'legs0_duration' ,
'legs1_duration','legs0_segments0_seatsAvailable','legs0_segments1_seatsAvailable','legs1_segments0_seatsAvailable','legs1_segments1_seatsAvailable',
'miniRules0_monetaryAmount' ,
'miniRules0_percentage',
'miniRules0_statusInfos' ,
'miniRules1_monetaryAmount' ,
'miniRules1_percentage' ,
'miniRules1_statusInfos' ,
'pricingInfo_isAccessTP' ,
'pricingInfo_passengerCount','legs0_segments0_baggageAllowance_quantity','legs0_segments1_baggageAllowance_quantity','legs1_segments0_baggageAllowance_quantity','legs1_segments1_baggageAllowance_quantity',
'legs0_segments0_arrivalTo_airport_iata','legs0_segments1_arrivalTo_airport_iata',
'legs1_segments0_arrivalTo_airport_iata','legs1_segments1_arrivalTo_airport_iata',
'legs0_segments0_arrivalTo_airport_city_iata','legs0_segments1_arrivalTo_airport_city_iata',
'legs1_segments0_arrivalTo_airport_city_iata','legs1_segments1_arrivalTo_airport_city_iata']

In [37]:
string_columns = [col_name for col_name, dtype in zip(x.columns, x.dtypes) if dtype == pl.Utf8]
print(string_columns)

['searchRoute', 'ranker_id', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments2_arrivalTo_airport_iata', 'legs0_segments3_arrivalTo_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments1_arrivalTo_airport_iata', 'legs1_segments2_arrivalTo_airport_iata', 'legs1_segments3_arrivalTo_airport_iata', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs0_segments2_arrivalTo_airport_city_iata', 'legs0_segments3_arrivalTo_airport_city_iata', 'legs1_segments0_arrivalTo_airport_city_iata', 'legs1_segments1_arrivalTo_airport_city_iata', 'legs1_segments2_arrivalTo_airport_city_iata', 'legs1_segments3_arrivalTo_airport_city_iata']


In [38]:
x = x.with_columns(
    pl.col('searchRoute', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments1_arrivalTo_airport_iata','legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs1_segments0_arrivalTo_airport_city_iata', 'legs1_segments1_arrivalTo_airport_city_iata', 'ranker_id').cast(pl.Categorical).cast(pl.UInt32)
)


In [39]:
x

Id,TotalPrice,searchRoute,corporateTariffCode,ranker_id,profileId,companyID,legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration,legs0_segments0_seatsAvailable,legs0_segments1_seatsAvailable,legs0_segments2_seatsAvailable,legs0_segments3_seatsAvailable,legs1_segments0_seatsAvailable,legs1_segments1_seatsAvailable,legs1_segments2_seatsAvailable,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,legs0_segments0_baggageAllowance_quantity,legs0_segments1_baggageAllowance_quantity,legs0_segments2_baggageAllowance_quantity,legs0_segments3_baggageAllowance_quantity,legs1_segments0_baggageAllowance_quantity,legs1_segments1_baggageAllowance_quantity,legs1_segments2_baggageAllowance_quantity,legs1_segments3_baggageAllowance_quantity,legs0_segments0_arrivalTo_airport_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments2_arrivalTo_airport_iata,legs0_segments3_arrivalTo_airport_iata,legs1_segments0_arrivalTo_airport_iata,legs1_segments1_arrivalTo_airport_iata,legs1_segments2_arrivalTo_airport_iata,legs1_segments3_arrivalTo_airport_iata,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments2_arrivalTo_airport_city_iata,legs0_segments3_arrivalTo_airport_city_iata,legs1_segments0_arrivalTo_airport_city_iata,legs1_segments1_arrivalTo_airport_city_iata,legs1_segments2_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_city_iata
i64,f64,u32,i64,u32,i64,i64,datetime[μs],datetime[μs],time,datetime[μs],datetime[μs],time,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,17254.0,0,null,13031,2087645,57323,2024-06-15 15:40:00,2024-06-15 16:20:00,02:40:00,2024-07-09 09:45:00,2024-07-09 14:20:00,02:35:00,9.0,null,null,null,9.0,null,null,null,null,null,null,null,null,null,1.0,1,1.0,null,null,null,1.0,null,null,null,41,null,null,null,73,null,null,null,41,null,null,null,73,null,null,null
1,53365.0,0,123,13031,2087645,57323,2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00,4.0,4.0,null,null,9.0,9.0,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,1.0,1.0,null,null,1.0,1.0,null,null,43,41,null,null,43,73,null,null,43,41,null,null,43,73,null,null
2,55935.0,0,null,13031,2087645,57323,2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00,4.0,4.0,null,null,9.0,9.0,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,1.0,1.0,null,null,1.0,1.0,null,null,43,41,null,null,43,73,null,null,43,41,null,null,43,73,null,null
3,84120.0,0,123,13031,2087645,57323,2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00,4.0,4.0,null,null,9.0,9.0,null,null,0.0,null,1.0,0.0,null,1.0,1.0,1,1.0,1.0,null,null,1.0,1.0,null,null,43,41,null,null,43,73,null,null,43,41,null,null,43,73,null,null
4,88310.0,0,null,13031,2087645,57323,2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00,4.0,4.0,null,null,9.0,9.0,null,null,0.0,null,1.0,0.0,null,1.0,1.0,1,1.0,1.0,null,null,1.0,1.0,null,null,43,41,null,null,43,73,null,null,43,41,null,null,43,73,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
18146427,36290.0,220,44,18450,3046852,54154,2024-11-05 20:50:00,2024-11-06 09:45:00,10:55:00,2024-11-09 19:25:00,2024-11-10 08:10:00,14:45:00,3.0,5.0,null,null,8.0,1.0,null,null,24000.0,null,1.0,0.0,null,0.0,0.0,1,0.0,0.0,null,null,0.0,0.0,null,null,57,63,null,null,81,62,null,null,57,63,null,null,81,135,null,null
18146428,33220.0,220,44,18450,3046852,54154,2024-11-05 00:20:00,2024-11-05 20:00:00,17:40:00,2024-11-09 21:10:00,2024-11-10 08:35:00,13:25:00,3.0,9.0,null,null,4.0,3.0,null,n

In [40]:
x = x.fill_nan(0.0)

In [41]:
x = x.with_columns(
    pl.col('TotalPrice','searchRoute','corporateTariffCode','ranker_id','profileId','companyID'
,'legs0_segments0_seatsAvailable','legs0_segments1_seatsAvailable','legs0_segments2_seatsAvailable',
'legs0_segments3_seatsAvailable','legs1_segments0_seatsAvailable','legs1_segments1_seatsAvailable'
,'legs1_segments2_seatsAvailable','legs1_segments3_seatsAvailable',
'miniRules0_monetaryAmount',
'miniRules0_percentage',
'miniRules0_statusInfos' ,
'miniRules1_monetaryAmount' ,
'miniRules1_percentage',
'miniRules1_statusInfos',
'pricingInfo_isAccessTP',
'pricingInfo_passengerCount','legs0_segments0_baggageAllowance_quantity',
'legs0_segments1_baggageAllowance_quantity','legs0_segments2_baggageAllowance_quantity',
'legs0_segments3_baggageAllowance_quantity','legs1_segments0_baggageAllowance_quantity',
'legs1_segments1_baggageAllowance_quantity','legs1_segments2_baggageAllowance_quantity',
'legs1_segments3_baggageAllowance_quantity'
,'legs0_segments0_arrivalTo_airport_iata','legs0_segments1_arrivalTo_airport_iata',
'legs0_segments2_arrivalTo_airport_iata','legs0_segments3_arrivalTo_airport_iata',
'legs1_segments0_arrivalTo_airport_iata','legs1_segments1_arrivalTo_airport_iata',
'legs1_segments2_arrivalTo_airport_iata','legs1_segments3_arrivalTo_airport_iata',
'legs0_segments0_arrivalTo_airport_city_iata','legs0_segments1_arrivalTo_airport_city_iata'
,'legs0_segments2_arrivalTo_airport_city_iata','legs0_segments3_arrivalTo_airport_city_iata',
'legs1_segments0_arrivalTo_airport_city_iata','legs1_segments1_arrivalTo_airport_city_iata',
'legs1_segments2_arrivalTo_airport_city_iata','legs1_segments3_arrivalTo_airport_city_iata').fill_nan(0).sum()
)

In [42]:
xt = dftt['Id','TotalPrice','searchRoute','corporateTariffCode','profileId','companyID',
'legs0_duration' ,
'legs1_duration','legs0_segments0_seatsAvailable','legs0_segments1_seatsAvailable','legs1_segments0_seatsAvailable','legs1_segments1_seatsAvailable',
'miniRules0_monetaryAmount' ,
'miniRules0_percentage',
'miniRules0_statusInfos' ,
'miniRules1_monetaryAmount' ,
'miniRules1_percentage' ,
'miniRules1_statusInfos' ,
'pricingInfo_isAccessTP' ,
'pricingInfo_passengerCount','legs0_segments0_baggageAllowance_quantity','legs0_segments1_baggageAllowance_quantity','legs1_segments0_baggageAllowance_quantity','legs1_segments1_baggageAllowance_quantity',
'legs0_segments0_arrivalTo_airport_iata','legs0_segments1_arrivalTo_airport_iata',
'legs1_segments0_arrivalTo_airport_iata','legs1_segments1_arrivalTo_airport_iata',
'legs0_segments0_arrivalTo_airport_city_iata','legs0_segments1_arrivalTo_airport_city_iata',
'legs1_segments0_arrivalTo_airport_city_iata','legs1_segments1_arrivalTo_airport_city_iata']

In [43]:
xt = xt.with_columns(
    pl.col('searchRoute', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments1_arrivalTo_airport_iata','legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs1_segments0_arrivalTo_airport_city_iata', 'legs1_segments1_arrivalTo_airport_city_iata', 'ranker_id').cast(pl.Categorical).cast(pl.UInt32)
)

In [44]:
groups = x.select('ranker_id')

# Trainig Model

In [45]:
! pip install XGBoost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.8/253.8 MB 2.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [46]:
import xgboost as xgb

In [48]:
!pip install -U optuna
import optuna
from sklearn.metrics import ndcg_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.4/584.4 kB 28.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    "learning_rate": 0.022641389657079056,
    "max_depth": 14,
    "min_child_weight": 2,
    "subsample": 0.8842234913702768,
    "colsample_bytree": 0.45840689146263086,
    "gamma": 3.3084297630544888,
    "lambda": 6.952586917313028,
    "alpha": 0.6395254133055179,
    'seed': 42,
    'n_jobs': -1,
}

In [50]:
x = x.drop('ranker_id',
'legs0_duration',
'legs1_duration')

In [52]:
groups= groups.group_by('ranker_id', maintain_order=True).agg(pl.len())['len'].to_numpy()

In [53]:
dtrain = xgb.DMatrix(x,group= groups, label=y)

In [ ]:
 bst = xgb.train(params, dtrain, num_boost_round=1000)

In [ ]:
import joblib
#Save the model
joblib.dump(bst, '/kaggle/working/my_model.pkl')

In [ ]:
groupt = xt.select('ranker_id')

In [ ]:
 groupt= groupt.group_by('ranker_id', maintain_order=True).agg(pl.len())['len'].to_numpy()

In [ ]:
xt = xt.drop('ranker_id',
'legs0_duration',
'legs1_duration')

In [ ]:
dtest = xgb.DMatrix(xt,group = groupt)

In [ ]:
predictions = bst.predict(dtest)

In [ ]:
submission = (
    dftt.select(['Id','ranker_id'])
    .with_columns(pl.Series('pred_score', predictions))
    .with_columns(
        pl.col('pred_score')
        .rank(method='ordinal', descending=True)
        .over('ranker_id')
        .cast(pl.Int32)
        .alias('selected')
    )
    .select(['Id','ranker_id', 'selected'])
)
submission.write_csv('/kaggle/working/submission.csv')

In [ ]:
import pandas as pd


In [ ]:
z = pd.read_csv('/kaggle/working/submission.csv')

In [ ]:
z.shape

In [ ]:
z.head()